In [1]:
import websocket
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
from code_ref import get_keys
from code_ref import findata
from code_ref import FinnhubRequestException
from code_ref import FinnhubAPIException
import time
import threading
%matplotlib inline
%load_ext autoreload
%autoreload 2
from ratelimit import limits
from ratelimiter import RateLimiter

In [2]:
keys = get_keys("/Users/augustingoudet/.secret/code_file") #Finnhub Key
api_key1 = keys ['finh2'] #api_key
company = "KER.PA" #Company Ticker
query = findata(api_key = api_key1)
rate_limiter = RateLimiter(max_calls=18, period=60)

In [3]:
url = "https://finnhub.io/api/v1/"

ext1 = "?symbol={}&token={}"
ext2 = "?symbol={}&metric=management&token={}"
ext3 = "?symbol={}&metric=margin&token={}"
ext4 = "?symbol={}&metric=growth&token={}"
ext5 = "?symbol={}&metric=price&token={}"
ext6 = "?symbol={}&metric=valuation&token={}"
ext7 = "?symbol={}&metric=financialStrength&token={}"
ext8 = "?symbol={}&metric=perShare&token={}"

profile ="stock/profile"
ceo_comp = "stock/ceo-compensation"
board = "stock/executive"
reco = "stock/recommendation"
price_target = "stock/price-target"
up_down = "stock/upgrade-downgrade"
peers = "stock/peers"
rev_est = "stock/revenue-estimate"
eps_est = "stock/eps-estimate"
ownership = "stock/investor-ownership"
metric= "stock/metric"
quote = "quote"
news = "news"
covid = "covid19"
transcript = "stock/transcripts"
divs = "stock/dividend?symbol={}&from=2019-02-01&to=2020-02-01"
split = "stock/split?symbol=AAPL&from=2010-02-01&to=2020-02-01"
inc = "stock/financials?symbol={}&statement=is&freq=annual"
bs = "stock/financials?symbol={}&statement=bs&freq=annual"
cfs = "stock/financials?symbol={}&statement=cf&freq=annual"

In [4]:
tickers = pd.read_csv('Ticker_list_1.csv')
tickers.drop(tickers[tickers['Exchange'] == 'PNK'].index, inplace = True)
drop = ['Argentina', 'Brazil', 'Estonia', 'Greece', 'Iceland', 'India', 'Indonesia', 'Latvia', 'Lithuania', 'Malaysia', 'Mexico', 'New Zealand', 'Qatar', 'Russia', 'Taiwan', 'Thailand', 'Turkey','China','Switzerland']
drop1 = ['MUN', 'STU', 'DUS', 'HAM', 'HAN', 'EUX', 'FRA', "ENX", 'BER', 'CNQ', 'VAN', 'IOB', 'KOE', 'PCX', 'OBB']
tickers.dropna(subset = ['Country'], inplace = True)

In [5]:
for values in drop:
    tickers.drop(tickers[tickers['Country'] == values].index, inplace = True)
for exch in drop1: 
    tickers.drop(tickers[tickers['Exchange'] == exch].index, inplace = True)

In [6]:
tickers['Category Name'].fillna(1, inplace = True)
drop2 = tickers[(tickers['Category Name'] == 1)].index
tickers.drop(drop2, inplace = True)

In [7]:
tickers.drop_duplicates(subset = 'Name', keep = 'first', inplace = True)
tickers.shape

(10867, 5)

In [8]:
tick = list(tickers.Ticker)


## Metrics: Valuation, Financial Strength

In [9]:
results = []

for items in tick: 
    with rate_limiter:
        data = url + metric + ext6.format(items, api_key1)
        req = requests.get(data).json()
        with open ('backup/valuation2.py', 'a+') as f:
            json.dump(req,f)
        results.append(req)
        m5 = json_normalize(results)
        m5.dropna(inplace=True)
m5.to_csv('data_api/valuation1.csv')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
